In [1]:
from astropy.io import fits
from reproject import reproject_interp
import numpy as np
import os

# --- Crop function (unchanged) ---
def _cropNanBorder(image, expected_shape):
    """
    Crop the nan border away such that the image fits the expected shape

    Parameters:
    - image (2D float array): The data to crop.
    - expected_shape (tuple): the expected shape to crop into.

    Returns:
    - Cropped (or padded) image matching expected_shape
    """
    mask = ~np.isnan(image)
    non_nan_rows = np.where(mask.any(axis=1))[0]
    non_nan_cols = np.where(mask.any(axis=0))[0]

    cropped_image = image[non_nan_rows.min():non_nan_rows.max() + 1,
                          non_nan_cols.min():non_nan_cols.max() + 1]
    current_shape = cropped_image.shape

    if current_shape != expected_shape:
        padded_image = np.full(expected_shape, np.nan)
        trim_rows = min(current_shape[0], expected_shape[0])
        trim_cols = min(current_shape[1], expected_shape[1])

        start_row = (expected_shape[0] - trim_rows) // 2
        start_col = (expected_shape[1] - trim_cols) // 2

        padded_image[start_row:start_row + trim_rows,
                     start_col:start_col + trim_cols] = cropped_image[:trim_rows, :trim_cols]

        return padded_image

    return cropped_image


# --- Paths ---
origDataPath = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\OriginalImages\ngc0628_F770W_JWST_Emission_starsub.fits"
alphaCOPath = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\PHANGS_alphaCO_conversion_factor_maps\NGC0628_alphaCO21_SL24_scaling.fits"
outputPath   = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\Reprojected\NGC0628_alphaCO21_reprojected.fits"

# --- Load Original Image for header + shape ---
with fits.open(origDataPath) as hdul:
    orig_data = hdul[0].data
    orig_header = hdul[0].header
    target_shape = orig_data.shape

# --- Load alphaCO Map ---
with fits.open(alphaCOPath) as hdul:
    alphaCO = np.array(hdul[0].data)
    alphaCO_header = hdul[0].header

# --- Reproject into OrigData frame ---
alphaCO_reproj, footprint = reproject_interp(
    (alphaCO, alphaCO_header),
    orig_header,
    shape_out=target_shape
)

# --- Crop to exact dimensions ---
alphaCO_reproj = _cropNanBorder(alphaCO_reproj, target_shape)

# --- Save as new FITS file ---
hdu = fits.PrimaryHDU(alphaCO_reproj, header=orig_header)
os.makedirs(os.path.dirname(outputPath), exist_ok=True)  # ensure dir exists
hdu.writeto(outputPath, overwrite=True)

print(f"Saved reprojected alphaCO map to {outputPath}")
nan_count = np.sum(alphaCO_reproj == np.nan)
print(f'Number of NaN pixels: {nan_count}')

Set DATE-AVG to '2022-07-17T12:01:53.586' from MJD-AVG.
Set DATE-END to '2022-07-17T12:54:46.016' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -37.754891 from OBSGEO-[XYZ].
Set OBSGEO-H to 1738895745.206 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Saved reprojected alphaCO map to C:\Users\jhoffm72\Documents\FilPHANGS\Data\Reprojected\NGC0628_alphaCO21_reprojected.fits
Number of NaN pixels: 0
